# Ejemplo de uso: Conexión a bases de datos SQL con `sql_connection`

Este notebook demuestra cómo conectarse a una base de datos existente usando el módulo OOP `sql_connection`.
El objetivo es realizar consultas de lectura sobre una base de datos real, sin crear ni modificar tablas.

Base de ejemplo: **toys_and_models.sqlite**

## Requisitos

Instala el paquete en modo editable desde la raíz del proyecto:

```bash
pip install -e .
```

Instala `pandas` si deseas mostrar los resultados como DataFrames:

In [ ]:
from sql_connection import get_connector
import os
try:
    import pandas as pd
except ImportError:
    pd = None

In [ ]:
# Ruta de la base de datos existente
db_path = os.path.abspath("examples/toys_and_models.sqlite")

if not os.path.exists(db_path):
    raise FileNotFoundError(f"No se encontró la base de datos: {db_path}")

# Crear conector y abrir conexión
conn = get_connector("sqlite", path=db_path)
with conn:
    print("✅ Conectado a:", conn.dsn_summary())
    print("Ping:", conn.ping())

In [ ]:
# Listar tablas de la base de datos
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"

if pd:
    display(conn.read_sql(query_tables))
else:
    cur = conn.conn.cursor()
    cur.execute(query_tables)
    print("Tablas:", cur.fetchall())
    cur.close()

In [ ]:
# Consultar los primeros clientes
query_customers = "SELECT customerNumber, customerName, country FROM customers LIMIT 10;"

if pd:
    display(conn.read_sql(query_customers))
else:
    cur = conn.conn.cursor()
    cur.execute(query_customers)
    for row in cur.fetchall():
        print(row)
    cur.close()

In [ ]:
# Consulta analítica: top 5 países por número de clientes
query_top_countries = """
SELECT country, COUNT(*) AS num_customers
FROM customers
GROUP BY country
ORDER BY num_customers DESC
LIMIT 5;
"""

if pd:
    display(conn.read_sql(query_top_countries))
else:
    cur = conn.conn.cursor()
    cur.execute(query_top_countries)
    print("Top países:", cur.fetchall())
    cur.close()

## Conclusiones

- El flujo OOP permite conectar con distintos motores (SQLite, PostgreSQL, Oracle) sin cambiar el código.
- `read_sql()` simplifica el análisis exploratorio con pandas.
- Este notebook puede servir como base para integraciones o dashboards SQL con Python.